In [1]:
# Célula 1: Setup, Lógica e Funções de BD (Final)

!pip install pandas
import sqlite3
import pandas as pd
import os
from datetime import date
from ipywidgets import VBox, Dropdown, FloatText, Button, Output
import ipywidgets as widgets # Importa o alias 'widgets' para a interface
from IPython.display import display, clear_output, Markdown
import numpy as np 

print("--- Setup Inicial Completo e Bibliotecas Importadas ---")

# =========================================================================
# CONSTANTES E DADOS DE EXEMPLO
# =========================================================================
CORTE_FREQUENCIA = 75
NOTA_APROVACAO_DIRETA = 7.0
NOTA_MINIMA_P3 = 4.0
NOTA_MINIMA_FINAL = 5.0
DB_NAME = 'diario_de_classe.db'
diario_de_classe = {
    "Alice": {
        "Português Instrumental": {
            "presencas": [{"data": "2025-09-01", "conteudo": "Revisão Gramatical", "status": 1}, {"data": "2025-09-08", "conteudo": "Análise de Texto", "status": 1}],
            "avaliacoes": {"P1": 9.0, "P2": 9.0, "P3": None}
        },
        "Inglês Instrumental": {
            "presencas": [{"data": "2025-09-02", "conteudo": "Skimming", "status": 1}, {"data": "2025-09-09", "conteudo": "Scanning", "status": 1}],
            "avaliacoes": {"P1": 8.0, "P2": 7.0, "P3": None}
        }
    },
    "Bruno": {
        "Português Instrumental": {
            "presencas": [{"data": "2025-09-01", "conteudo": "Revisão Gramatical", "status": 1}, {"data": "2025-09-08", "conteudo": "Análise de Texto", "status": 0}],
            "avaliacoes": {"P1": 6.0, "P2": 6.0, "P3": 8.0}
        },
        "Inglês Instrumental": {
            "presencas": [{"data": "2025-09-02", "conteudo": "Skimming", "status": 0}, {"data": "2025-09-09", "conteudo": "Scanning", "status": 1}],
            "avaliacoes": {"P1": 5.0, "P2": 4.0, "P3": 6.0}
        }
    },
    "Carol": {
        "Português Instrumental": {
            "presencas": [{"data": "2025-09-01", "conteudo": "Revisão Gramatical", "status": 1}, {"data": "2025-09-08", "conteudo": "Análise de Texto", "status": 1}],
            "avaliacoes": {"P1": 5.0, "P2": 5.0, "P3": None}
        },
        "Inglês Instrumental": {
            "presencas": [{"data": "2025-09-02", "conteudo": "Skimming", "status": 0}, {"data": "2025-09-09", "conteudo": "Scanning", "status": 0}],
            "avaliacoes": {"P1": 10.0, "P2": 10.0, "P3": None}
        }
    },
}

# =========================================================================
# FUNÇÕES PRINCIPAIS DE LÓGICA E BD
# =========================================================================
def get_nota_valida(nota):
    """Garante que None ou np.nan sejam tratados como 0.0 para cálculo parcial, mas mantém o None para a lógica de P3."""
    if nota is None:
        return 0.0
    try:
        if np.isnan(nota):
            return 0.0
    except:
        pass # Ignora se não for um float (ex: se for um int ou string)
    return float(nota)

def calcular_media_final(avaliacoes):
    p1_val = avaliacoes.get("P1")
    p2_val = avaliacoes.get("P2")
    p3_val = avaliacoes.get("P3")

    # Notas válidas (0.0 se for None/nan) para o cálculo parcial
    p1 = get_nota_valida(p1_val)
    p2 = get_nota_valida(p2_val)
    
    p3 = p3_val # Mantém o valor original (pode ser None) para a lógica de situação

    media_parcial = (p1 + p2) / 2
    nota_final = media_parcial
    situacao_nota = ""
    
    # Resto da lógica...
    if media_parcial >= NOTA_APROVACAO_DIRETA:
        situacao_nota = "APROVADO POR MÉDIA"
    elif media_parcial >= NOTA_MINIMA_P3:
        # AQUI usamos p3_val direto para ver se a nota foi lançada
        if p3_val is None:
            situacao_nota = "PENDENTE (AGUARDANDO P3)"
        else:
            p3_calc = get_nota_valida(p3_val) # Pega o valor 0.0 se for NaN/None
            media_final_com_p3 = (media_parcial + p3_calc) / 2
            nota_final = media_final_com_p3
            if nota_final >= NOTA_MINIMA_FINAL:
                situacao_nota = "APROVADO APÓS P3"
            else:
                situacao_nota = "REPROVADO POR NOTA"
    else: 
        situacao_nota = "REPROVADO DIRETO"
        
    return nota_final, situacao_nota, media_parcial
def lancar_aula_e_frequencia(id_disciplina, data_aula, conteudo):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    id_turma_padrao = 1
    try:
        cursor.execute("""INSERT INTO Aulas (id_turma, id_disciplina, data_aula, conteudo_lecionado) VALUES (?, ?, ?, ?)""", (id_turma_padrao, id_disciplina, data_aula, conteudo))
        conn.commit()
        id_aula = cursor.lastrowid
        cursor.execute("SELECT id_aluno FROM Alunos")
        alunos_ids = [row[0] for row in cursor.fetchall()]
        registros_frequencia = [(id_aula, id_aluno, 1) for id_aluno in alunos_ids]
        cursor.executemany("""INSERT INTO Frequencia (id_aula, id_aluno, presente) VALUES (?, ?, ?)""", registros_frequencia)
        conn.commit()
        print(f"✅ Aula de {conteudo} em {data_aula} lançada (ID: {id_aula}). Todos marcados como Presentes.")
    except Exception as e:
        print(f"❌ Erro ao lançar aula: {e}")
    finally:
        conn.close()

def inserir_nota_no_db(id_aluno, id_disciplina, tipo_avaliacao, valor_nota):
    if valor_nota is None or valor_nota < 0 or valor_nota > 10.0:
        print("⚠️ Erro: Insira um valor de nota válido (0.0 a 10.0).")
        return
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""REPLACE INTO Notas (id_aluno, id_disciplina, tipo_avaliacao, valor_nota) VALUES (?, ?, ?, ?)""", (id_aluno, id_disciplina, tipo_avaliacao, valor_nota))
        conn.commit()
        print(f"✅ Nota {tipo_avaliacao} ({valor_nota:.1f}) inserida/atualizada para o Aluno {id_aluno} na Disciplina {id_disciplina}.")
    except Exception as e:
        print(f"❌ Erro ao inserir nota: {e}")
    finally: conn.close()

def carregar_ids():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    alunos_db = {nome: id_a for id_a, nome in cursor.execute("SELECT id_aluno, nome FROM Alunos").fetchall()}
    disciplinas_db = {nome: id_d for id_d, nome in cursor.execute("SELECT id_disciplina, nome_disciplina FROM Disciplinas").fetchall()}
    conn.close()
    return alunos_db, disciplinas_db
    
def obter_frequencia_por_aula(id_disciplina, data_aula):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    id_turma_padrao = 1
    
    cursor.execute("""
        SELECT id_aula FROM Aulas WHERE id_turma = ? AND id_disciplina = ? AND data_aula = ?
    """, (id_turma_padrao, id_disciplina, data_aula))
    result = cursor.fetchone()
    
    if not result:
        conn.close()
        return None, "❌ Aula não encontrada para essa data/disciplina."
        
    id_aula = result[0]
    
    df = pd.read_sql_query(f"""
        SELECT 
            A.nome AS Aluno, 
            F.id_frequencia,
            F.presente 
        FROM Frequencia F
        JOIN Alunos A ON F.id_aluno = A.id_aluno
        WHERE F.id_aula = {id_aula}
        ORDER BY A.nome;
    """, conn)
    conn.close()
    
    df['Status Atual'] = df['presente'].apply(lambda x: 'PRESENTE ✅' if x == 1 else 'FALTA 🚫')
    df['Opção'] = df['id_frequencia'].astype(str) + ' - ' + df['Aluno']
    
    return df, id_aula
    
def atualizar_status_frequencia(id_frequencia, novo_status):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            UPDATE Frequencia SET presente = ? WHERE id_frequencia = ?
        """, (novo_status, id_frequencia))
        conn.commit()
        return f"✅ Status de Presença Atualizado! (ID Frequência: {id_frequencia})"
    except Exception as e:
        return f"❌ Erro ao atualizar frequência: {e}"
    finally:
        conn.close()

--- Setup Inicial Completo e Bibliotecas Importadas ---


In [2]:
# Célula 2: Criação e População do Banco de Dados (Limpo para SQLite)

def criar_e_popular_sqlite():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # 1. DELETAR TABELAS ANTIGAS PARA GARANTIR ESTRUTURA CORRETA
    cursor.execute("DROP TABLE IF EXISTS Frequencia")
    cursor.execute("DROP TABLE IF EXISTS Notas")
    cursor.execute("DROP TABLE IF EXISTS Aulas")
    cursor.execute("DROP TABLE IF EXISTS Alunos")
    cursor.execute("DROP TABLE IF EXISTS Disciplinas")
    cursor.execute("DROP TABLE IF EXISTS Turmas")
    conn.commit()

    # 2. CRIAÇÃO DAS TABELAS COM ESTRUTURA CORRETA (APENAS SQLITE)
    cursor.execute('''CREATE TABLE Alunos (id_aluno INTEGER PRIMARY KEY, nome TEXT NOT NULL, matricula TEXT UNIQUE NOT NULL);''')
    cursor.execute('''CREATE TABLE Disciplinas (id_disciplina INTEGER PRIMARY KEY, nome_disciplina TEXT UNIQUE NOT NULL);''')
    cursor.execute('''CREATE TABLE Turmas (id_turma INTEGER PRIMARY KEY, nome_turma TEXT NOT NULL, ano_letivo INTEGER NOT NULL);''')
    cursor.execute('''CREATE TABLE Aulas (id_aula INTEGER PRIMARY KEY, id_turma INTEGER, id_disciplina INTEGER, data_aula DATE NOT NULL, conteudo_lecionado TEXT, FOREIGN KEY (id_turma) REFERENCES Turmas(id_turma), FOREIGN KEY (id_disciplina) REFERENCES Disciplinas(id_disciplina));''')
    cursor.execute('''CREATE TABLE Notas (id_nota INTEGER PRIMARY KEY, id_aluno INTEGER, id_disciplina INTEGER, tipo_avaliacao TEXT NOT NULL, valor_nota REAL NOT NULL, UNIQUE(id_aluno, id_disciplina, tipo_avaliacao), FOREIGN KEY (id_aluno) REFERENCES Alunos(id_aluno), FOREIGN KEY (id_disciplina) REFERENCES Disciplinas(id_disciplina));''')
    cursor.execute('''CREATE TABLE Frequencia (id_frequencia INTEGER PRIMARY KEY, id_aula INTEGER, id_aluno INTEGER, presente BOOLEAN NOT NULL, UNIQUE(id_aula, id_aluno), FOREIGN KEY (id_aula) REFERENCES Aulas(id_aula), FOREIGN KEY (id_aluno) REFERENCES Alunos(id_aluno));''')
    conn.commit()

    # 3. POPULANDO OS DADOS
    aluno_map = {}; disciplina_map = {}; id_turma_padrao = 1
    cursor.execute("REPLACE INTO Turmas (id_turma, nome_turma, ano_letivo) VALUES (?, ?, ?)", (id_turma_padrao, "Exemplo 2025/1", 2025))
    disciplinas_list = ["Português Instrumental", "Inglês Instrumental"]
    for i, disc in enumerate(disciplinas_list): cursor.execute("REPLACE INTO Disciplinas (id_disciplina, nome_disciplina) VALUES (?, ?)", (i+1, disc))
    cursor.execute("SELECT id_disciplina, nome_disciplina FROM Disciplinas")
    for id_disc, nome_disc in cursor.fetchall(): disciplina_map[nome_disc] = id_disc
    
    alunos_list = list(diario_de_classe.keys())
    for i, aluno in enumerate(alunos_list): 
        cursor.execute("REPLACE INTO Alunos (id_aluno, nome, matricula) VALUES (?, ?, ?)", (i+1, aluno, f"MAT{2025000 + i + 1}"))
    cursor.execute("SELECT id_aluno, nome FROM Alunos")
    for id_aluno, nome_aluno in cursor.fetchall(): aluno_map[nome_aluno] = id_aluno

    for nome_aluno, disciplinas_aluno in diario_de_classe.items():
        id_aluno = aluno_map[nome_aluno]
        for nome_disciplina, dados in disciplinas_aluno.items():
            id_disciplina = disciplina_map[nome_disciplina]
            
            for aula_data in dados["presencas"]:
                data = aula_data["data"]; conteudo = aula_data["conteudo"]; status_presenca = aula_data["status"]
                cursor.execute("""REPLACE INTO Aulas (id_turma, id_disciplina, data_aula, conteudo_lecionado) VALUES (?, ?, ?, ?)""", (id_turma_padrao, id_disciplina, data, conteudo))
                cursor.execute("""SELECT id_aula FROM Aulas WHERE id_turma = ? AND id_disciplina = ? AND data_aula = ?""", (id_turma_padrao, id_disciplina, data))
                result = cursor.fetchone()
                if result:
                    id_aula = result[0]
                    cursor.execute("""REPLACE INTO Frequencia (id_aula, id_aluno, presente) VALUES (?, ?, ?)""", (id_aula, id_aluno, status_presenca))
            
            for tipo_avaliacao, valor_nota in dados["avaliacoes"].items():
                if valor_nota is not None: cursor.execute("""REPLACE INTO Notas (id_aluno, id_disciplina, tipo_avaliacao, valor_nota) VALUES (?, ?, ?, ?)""", (id_aluno, id_disciplina, tipo_avaliacao, valor_nota))
    
    conn.commit()
    conn.close()
    print(f"✅ Banco de Dados '{DB_NAME}' recriado e populado (incluindo Carol).")

criar_e_popular_sqlite()

✅ Banco de Dados 'diario_de_classe.db' recriado e populado (incluindo Carol).


In [10]:
# Célula 3: Relatório Final e Formulários (Execução)

def gerar_relatorio_final_completo():
    """Executa a consulta SQL unificada, aplica a lógica Python e imprime o boletim."""
    try:
        conn = sqlite3.connect(DB_NAME)
        query_sql_completa = """
        SELECT A.nome AS "Aluno", D.nome_disciplina AS "Disciplina", 
            MAX(CASE WHEN N.tipo_avaliacao = 'P1' THEN N.valor_nota ELSE NULL END) AS "P1",
            MAX(CASE WHEN N.tipo_avaliacao = 'P2' THEN N.valor_nota ELSE NULL END) AS "P2",
            MAX(CASE WHEN N.tipo_avaliacao = 'P3' THEN N.valor_nota ELSE NULL END) AS "P3",
            COUNT(CASE WHEN F.presente = 1 THEN 1 ELSE NULL END) AS "Total_Presencas",
            COUNT(AU.id_aula) AS "Total_Aulas"
        FROM Alunos A CROSS JOIN Disciplinas D 
        LEFT JOIN Notas N ON A.id_aluno = N.id_aluno AND D.id_disciplina = N.id_disciplina
        LEFT JOIN Aulas AU ON D.id_disciplina = AU.id_disciplina
        LEFT JOIN Frequencia F ON A.id_aluno = F.id_aluno AND AU.id_aula = F.id_aula
        GROUP BY A.nome, D.nome_disciplina;
        """
        df_relatorio = pd.read_sql_query(query_sql_completa, conn)

    except Exception as e:
        display(widgets.HTML(f"<p style='color:red;'>❌ <b>ERRO FATAL</b> na consulta SQL/Pandas. Mensagem: {e}</p>"))
        return

    resultados_finais = []
    for index, row in df_relatorio.iterrows():
        total_aulas = row['Total_Aulas'] or 0; total_presencas = row['Total_Presencas'] or 0
        frequencia_percentual = (total_presencas / total_aulas * 100) if total_aulas > 0 else 0
        avaliacoes = {"P1": row['P1'], "P2": row['P2'], "P3": row['P3']}
        nota_final, situacao_nota, media_parcial = calcular_media_final(avaliacoes)
        situacao_frequencia = "REPROVADO POR FALTA" if frequencia_percentual < CORTE_FREQUENCIA else "APROVADO POR FREQUÊNCIA"

        if situacao_frequencia.startswith("REPROVADO") or situacao_nota.startswith("REPROVADO"):
            situacao_final = "REPROVADO GERAL 🔴"
        elif situacao_nota.startswith("PENDENTE"):
            situacao_final = "PENDENTE ⚠️"
        else:
            situacao_final = "APROVADO GERAL 🟢"

        resultados_finais.append({
            "Aluno": row['Aluno'], "Disciplina": row['Disciplina'],
            "P1": f"{row['P1']:.1f}" if pd.notna(row['P1']) else '-',
            "P2": f"{row['P2']:.1f}" if pd.notna(row['P2']) else '-',
            "P3": f"{row['P3']:.1f}" if pd.notna(row['P3']) else '-',
            "Frequência (%)": f"{frequencia_percentual:.1f}",
            "Nota Final": f"{nota_final:.1f}",
            "Situação Final": situacao_final
        })

    if not resultados_finais: display(widgets.HTML("<b>Nenhum dado encontrado para o relatório.</b>")); return
    display(widgets.HTML("<h2>📋 Relatório Final Consolidado (Tabela Visual)</h2>"))
    df_final = pd.DataFrame(resultados_finais)
    display(df_final.set_index(["Aluno", "Disciplina"]))


# --- VARIÁVEIS DE ESTADO E INICIALIZAÇÃO DE IDS ---

alunos_map, disciplinas_map = carregar_ids()
global df_frequencia_atual
df_frequencia_atual = pd.DataFrame() 

# =========================================================================
# FUNÇÕES DE CALLBACK (Ações dos Botões)
# =========================================================================

def on_button_click(b): # Ação do botão de NOTAS
    with output_area: 
        clear_output(wait=True)
        id_aluno = aluno_widget.value
        id_disciplina = disciplina_widget.value
        tipo = avaliacao_widget.value
        valor = nota_widget.value
        inserir_nota_no_db(id_aluno, id_disciplina, tipo, valor)
        display(widgets.HTML("<hr><h4>Relatório ATUALIZADO</h4>"))
        gerar_relatorio_final_completo()

def on_lancar_aula_click(b): # Ação do botão de LANÇAR AULA
    with output_aula:
        clear_output(wait=True)
        id_disciplina = disciplina_aula_widget.value
        data = data_widget.value
        conteudo = conteudo_widget.value
        
        lancar_aula_e_frequencia(id_disciplina, data, conteudo)
        
        display(widgets.HTML("<hr><h4>Relatório ATUALIZADO (Frequência)</h4>"))
        gerar_relatorio_final_completo()

def on_carregar_chamada_click(b): # Ação do botão CARREGAR CHAMADA
    with output_chamada:
        clear_output(wait=True)
        id_disciplina = disciplina_consulta_widget.value
        data = data_consulta_widget.value
        
        global df_frequencia_atual
        df, id_aula = obter_frequencia_por_aula(id_disciplina, data)
        
        if df is None:
            display(widgets.HTML(f"<h3>❌ ERRO: {id_aula}</h3>"))
            # Garante que o dropdown de ajuste de alunos fique vazio
            aluno_frequencia_widget.options = {} 
            return

        df_frequencia_atual = df
        
        # 1. Cria o dicionário {Nome do Aluno: ID_Frequencia}
        opcoes_dropdown = {row['Aluno']: row['id_frequencia'] for index, row in df.iterrows()}
        
        # 2. Define as opções e o valor inicial (Nome do aluno aparece corretamente)
        aluno_frequencia_widget.index = None # Reseta o estado
        aluno_frequencia_widget.options = opcoes_dropdown 
        aluno_frequencia_widget.value = list(opcoes_dropdown.values())[0] if opcoes_dropdown else None
        
        display(widgets.HTML(f"<h3>✅ Chamada Carregada (Aula ID: {id_aula})</h3>"))
        display(df[['Aluno', 'Status Atual']])
        display(widgets.HTML("<hr><i>Use o passo 2 para mudar o status de um aluno.</i>"))

def on_atualizar_frequencia_click(b): # Ação do botão SALVAR ALTERAÇÃO
    with output_chamada:
        if aluno_frequencia_widget.value is None:
            display(widgets.HTML("⚠️ Por favor, carregue a chamada primeiro e selecione um aluno."))
            return
            
        id_frequencia_registro = aluno_frequencia_widget.value
        novo_status = status_novo_widget.value
        
        resultado = atualizar_status_frequencia(id_frequencia_registro, novo_status)
        display(widgets.HTML(resultado))
        
        # Recarrega a tabela de chamada e o relatório geral
        on_carregar_chamada_click(None) 
        display(widgets.HTML("<hr><h4>Relatório GERAL Atualizado</h4>"))
        gerar_relatorio_final_completo()

# =========================================================================
# DEFINIÇÃO DOS WIDGETS E MONTAGEM DA INTERFACE
# =========================================================================

# Formulário de AULAS e FREQUÊNCIA
output_aula = widgets.Output()
data_widget = widgets.Text(value=str(date.today()), description='Data (YYYY-MM-DD):')
conteudo_widget = widgets.Text(description='Conteúdo da Aula:')
disciplina_aula_widget = widgets.Dropdown(options=disciplinas_map, value=list(disciplinas_map.values())[0], description='Disciplina:')
botao_lancar_aula = widgets.Button(description="Lançar Aula e Marcar Todos Presentes")
botao_lancar_aula.on_click(on_lancar_aula_click)
formulario_aula = widgets.VBox([disciplina_aula_widget, data_widget, conteudo_widget, botao_lancar_aula, output_aula])

# PAINEL DE CHAMADA (REVISÃO DE FALTAS)
output_chamada = widgets.Output()
data_consulta_widget = widgets.Text(value=str(date.today()), description='Data da Aula:')
disciplina_consulta_widget = widgets.Dropdown(options=disciplinas_map, value=list(disciplinas_map.values())[0], description='Disciplina:')
botao_carregar_chamada = widgets.Button(description="1. Carregar Chamada da Aula")
botao_carregar_chamada.on_click(on_carregar_chamada_click)

# Bloco 3: Inicialização do PAINEL DE CHAMADA (REVISÃO DE FALTAS)

# ... (código de definição dos outros widgets)

# INICIALIZAÇÃO CORRIGIDA: Agora ele usa alunos_map na inicialização
aluno_frequencia_widget = widgets.Dropdown(
    # Usa o mapa de alunos {Nome: ID_Aluno}
    options=alunos_map,
    # Define o valor inicial como o ID do primeiro aluno
    value=list(alunos_map.values())[0] if alunos_map else None,
    description='Aluno para Ajuste:' 
)

# ... (restante do código do Painel de Chamada)

# CORREÇÃO DA INICIALIZAÇÃO: Inicializa com Nomes dos Alunos
aluno_frequencia_widget = widgets.Dropdown(
    options={nome: id_a for nome, id_a in alunos_map.items()}, 
    value=list(alunos_map.values())[0] if alunos_map else None,
    description='Aluno para Ajuste:'
)
status_novo_widget = widgets.Dropdown(options=[('PRESENTE', 1), ('FALTA', 0)], value=1, description='Novo Status:') 
botao_atualizar_frequencia = widgets.Button(description="2. Salvar Alteração")
botao_atualizar_frequencia.on_click(on_atualizar_frequencia_click)

painel_consulta = widgets.VBox([disciplina_consulta_widget, data_consulta_widget, botao_carregar_chamada])
painel_atualizacao = widgets.VBox([aluno_frequencia_widget, status_novo_widget, botao_atualizar_frequencia])

# Formulário de NOTAS
output_area = widgets.Output()
aluno_widget = widgets.Dropdown(options=alunos_map, value=list(alunos_map.values())[0], description='Aluno(a):')
disciplina_widget = widgets.Dropdown(options=disciplinas_map, value=list(disciplinas_map.values())[0], description='Disciplina:')
avaliacao_widget = widgets.Dropdown(options=['P1', 'P2', 'P3'], value='P1', description='Avaliação:')
nota_widget = widgets.FloatText(value=7.0, description='Nota (0-10):', min=0.0, max=10.0)
botao_inserir = widgets.Button(description="Inserir/Atualizar Nota e Gerar Relatório")
botao_inserir.on_click(on_button_click)
formulario = widgets.VBox([aluno_widget, disciplina_widget, avaliacao_widget, nota_widget, botao_inserir, output_area])

# --- EXIBIÇÃO FINAL ---

display(widgets.HTML("<h2>🗓️ Lançamento de Aulas e Frequência</h2>"))
display(formulario_aula)

display(widgets.HTML("<h2>📋 Painel de Chamada e Ajuste de Faltas</h2>"))
display(widgets.HBox([painel_consulta, painel_atualizacao]))
display(output_chamada)

display(widgets.HTML("<h2>🖊️ Inserção Visual de Notas (`ipywidgets`)</h2>"))
display(formulario)

# Exibição Inicial do Relatório
print("\n--- Relatório Inicial ---")
gerar_relatorio_final_completo()

HTML(value='<h2>🗓️ Lançamento de Aulas e Frequência</h2>')

HTML(value='<h2>📋 Painel de Chamada e Ajuste de Faltas</h2>')

Output()

HTML(value='<h2>🖊️ Inserção Visual de Notas (`ipywidgets`)</h2>')


--- Relatório Inicial ---


HTML(value='<h2>📋 Relatório Final Consolidado (Tabela Visual)</h2>')

P1    P2   P3 Frequência (%) Nota Final  \
Aluno Disciplina                                                          
Alice Inglês Instrumental      8.0   7.0    -           33.3        7.5   
      Português Instrumental   9.0   9.0    -           33.3        9.0   
Bruno Inglês Instrumental      5.0   4.0  6.0           16.7        5.2   
      Português Instrumental   6.0   6.0  8.0           16.7        7.0   
Carol Inglês Instrumental     10.0  10.0    -            0.0       10.0   
      Português Instrumental   5.0   5.0    -           33.3        5.0   

                                 Situação Final  
Aluno Disciplina                                 
Alice Inglês Instrumental     REPROVADO GERAL 🔴  
      Português Instrumental  REPROVADO GERAL 🔴  
Bruno Inglês Instrumental     REPROVADO GERAL 🔴  
      Português Instrumental  REPROVADO GERAL 🔴  
Carol Inglês Instrumental     REPROVADO GERAL 🔴  
      Português Instrumental  REPROVADO GERAL 🔴